In [1]:
import numpy as np
import torch
import os
import torch.nn as nn
import torchvision.datasets as ds
import torchvision.transforms as tf
from torch.utils.data import DataLoader as dl
import torch.nn.functional as f
import torch.optim as optim

Class for preprocessing image data

In [192]:
class ImgPreprocess:
    def __init__(self):
        self.path=os.getcwd()
        #print('cur dir:', self.path)
    def processRawImg(self,dir_nm):
        img_tf=tf.Compose([tf.RandomResizedCrop(256), tf.ToTensor()])
        
        img_dir=ds.ImageFolder(os.path.join(self.path,dir_nm), transform=img_tf)
        return img_dir
 
    def arrangeImgDir(self,tr_dir):
      ''' This function arranges the images in a pattern of folder structure such that the ImageFolder class can read the classes'''
      ori=os.getcwd()
      print('Current directory:', ori)
      if 'train' not in ori:
        os.chdir(tr_dir)
        aft=os.getcwd()
        print('Changed directory:',aft)
      else:
        aft=ori
      if os.path.exists(os.path.join(aft,'Cat'))==False:
         !mkdir Cat
      if os.path.exists(os.path.join(aft,'Dog'))==False:
         !mkdir Dog
      for f in os.listdir():
        if f[-4:]=='.jpg':
          if 'cat' in f:
            fn=os.path.join(aft,f)
            #print('Moving file:',fn)
            shutil.move(fn,os.path.join(aft,'Cat'))
          else:
            fn=os.path.join(aft,f)
            #print('Moving file:',fn)
            shutil.move(fn,os.path.join(aft,'Dog'))
      print('len of cat folder:',len(os.listdir(os.path.join(aft, 'Cat'))))
      print('len of Dog folder:',len(os.listdir(os.path.join(aft, 'Dog'))))
      print(os.listdir(aft))
      #os.chdir(ori)
      %cd ..
      print('Revert to original directory:',os.getcwd())

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Define the model and it's methods

In [6]:
class Model(nn.Module):
    ''' define the architecture of cnn. It has 4 convolutional layers
        followed by pooling layers and 3 fully connected layers.'''
    def __init__(self):
        super(Model,self).__init__()
        self.conv1=nn.Conv2d(3,8,3,padding=1)
        self.conv2=nn.Conv2d(8,16,3,padding=1)
        self.conv3=nn.Conv2d(16,32,3,padding=1)
        self.conv4=nn.Conv2d(32,64,3,padding=1)
        self.pool=nn.MaxPool2d(2,2)
        self.linear1=nn.Linear(16*16*64, 128)
        self.linear2=nn.Linear(128,64)
        self.linear3=nn.Linear(64,3)
 
    def forward(self, inp):
      for conv in [self.conv1, self.conv2, self.conv3, self.conv4]:
        inp=self.pool(f.relu(conv(inp)))
        
      #print(inp.shape)
      inp=inp.view(-1,inp.shape[1]*inp.shape[2]*inp.shape[3])
      #for lin in [self.linear1, self.linear2]:
      #print(inp.shape)
      inp=f.relu(self.linear1(inp))
      inp=f.relu(self.linear2(inp))
      inp=self.linear3(inp)
      return inp

In [7]:
 def train(model, d_ldr, epochs, criterion,optim):
        tr_loss=[]
        model.train()
        for e in range(epochs):
          print( 'epoch:',e)
          for data, lbl in d_ldr:
                 data=data.to(dev)
                 lbl=lbl.to(dev)
                 optim.zero_grad()
                 out=model(data)
                 #print(out.shape)
                 #print(lbl.long().shape)
                 loss=criterion(out, lbl)
                 loss.backward()
                 optim.step()
                 del out, data, lbl
                 torch.cuda.empty_cache()
                 tr_loss.append(loss.item())
        cp={'state_dict': model.state_dict()}
        torch.save(cp, 'Checkpoint.pth')     
        return tr_loss, model
 
def eval(model, tst_ldr,epoch, criterion):
         tst_loss=[]
         true=0
         total=0
         model.eval()
         for data, lbl in tst_ldr:
            data=data.to(dev)
            lbl=lbl.to(dev)
            out=model(data)
            loss =criterion(out, lbl.long())
            _, pred=torch.max(out.data,1)
            total+=lbl.size(0)
            true+=(pred==lbl).sum().item()
            acc=100*true/total
         return tst_loss,acc

In [8]:
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [146]:
!unzip -q '/content/drive/My Drive/ComputerVision/CatvsDogs/train.zip'

In [128]:
tr_dir_nm='train'

In [109]:
import shutil

In [179]:
ipp=ImgPreprocess()

In [157]:
 ipp.arrangeImgDir(tr_dir_nm)

Current directory: /content
Changed directory: /content/train
len of cat folder: 12500
len of Dog folder: 12500
['Cat', 'Dog']
/content
Revert to original directory: /content


In [158]:
tr_img=ipp.processRawImg(tr_dir_nm)

In [159]:
tr_ldr=dl(tr_img, batch_size=50, shuffle=True)

In [160]:
for d, l in tr_ldr:
   print(d.shape)
   print(l.shape)
   print(l)
   break

torch.Size([50, 3, 256, 256])
torch.Size([50])
tensor([1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
        1, 1])


In [162]:
len(tr_ldr)

500

In [163]:
m=Model().to(dev)

In [166]:
criterion=nn.CrossEntropyLoss()
opt=optim.Adam(m.parameters(),lr=.01)

In [167]:
tr_loss, model=train(m, tr_ldr,20, criterion, opt)

epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18
epoch: 19


In [168]:
tr_loss

[1.122840166091919,
 0.7424131631851196,
 2.0759923458099365,
 0.8286415934562683,
 0.8655161261558533,
 0.7310003638267517,
 0.6909639835357666,
 1.4414427280426025,
 0.8776746988296509,
 0.7126139998435974,
 0.7364314198493958,
 0.6968149542808533,
 0.7739751935005188,
 0.796946108341217,
 0.7462708353996277,
 0.7316426038742065,
 0.6829249858856201,
 0.6836405992507935,
 0.7913570404052734,
 0.7325670719146729,
 0.6382757425308228,
 0.7231436371803284,
 0.6927446126937866,
 0.698265016078949,
 0.7158000469207764,
 0.6944454312324524,
 0.6847926378250122,
 0.685539960861206,
 0.7158467173576355,
 0.6954891085624695,
 0.7058678269386292,
 0.6998097896575928,
 0.6954508423805237,
 0.6974853277206421,
 0.6921059489250183,
 0.6923792362213135,
 0.6939088702201843,
 0.7178467512130737,
 0.6909623146057129,
 0.6962274312973022,
 0.6987165212631226,
 0.6930474042892456,
 0.6905046105384827,
 0.7014373540878296,
 0.6908012628555298,
 0.6939433813095093,
 0.6884098649024963,
 0.69555503129959

In [190]:
!unzip -q '/content/drive/My Drive/ComputerVision/CatvsDogs/test1.zip'

In [187]:
tst_dir_nm='test1'

In [188]:
 ipp.arrangeImgDir(tst_dir_nm)

Current directory: /content
Changed directory: /content/test1
len of cat folder: 0
len of Dog folder: 12500
['Cat', 'Dog']
/content
Revert to original directory: /content


In [ ]:
 tst_img=ipp.processRawImg(tst_dir_nm)

In [ ]:
 tst_ldr=dl(tst_img, batch_size=50, shuffle=True)

In [ ]:
 for d, l in tst_ldr:
   print(d.shape)
   print(l.shape)
   print(l)
   break

In [ ]:
 tst_loss, acc=eval(model, tst_ldr,20, criterion)

In [ ]:
print('test accuracy:',acc)

In [194]:
#%rm -rf '/content/test1'